In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import re

import warnings
warnings.filterwarnings("ignore")

# *Lendo, organizando e gerando as bases*

In [28]:
# lendo episodios e renomeando colunas
eps = pd.read_excel('data/kaggle/the_office_episodes.xlsx')
eps.rename(columns={'season':'temporada', 
                    'episode_num_in_season':'episodio',
                    'episode_num_overall':'episodeo_num_geral',
                    'title':'titulo',
                    'directed_by':'dirigido_por',
                    
                    'written_by':'escrito_por',
                    'original_air_date':'data_de_estreia',
                    'prod_code':'codigo',
                    'us_viewers':'telespectadores_USA'},
            inplace=True)

# lendo imdb e renomando colunas
imdb = pd.read_excel('data/kaggle/the_office_imdb.xlsx')
imdb.rename(columns={'season':'temporada', 
                    'episode_num':'episodio',
                    'title':'titulo',
                    'original_air_date':'data_de_estreia',
                    'imdb_rating':'nota_imdb',
                    'total_votes':'numero_de_votos',
                    'desc':'descricao'},
            inplace=True)

# juntando as informações e deletando colunas
eps = eps.merge(imdb, on='titulo', suffixes=('', '_delme'))
eps = eps[[c for c in eps.columns if not c.endswith('_delme')]]
eps = eps[['temporada', 'episodio', 'episodeo_num_geral', 
           'titulo', 'descricao', 'dirigido_por', 'escrito_por',
           'data_de_estreia', 'codigo', 'telespectadores_USA', 
           'nota_imdb', 'numero_de_votos']]
del imdb

# lendo series e renomeado colunas
series = pd.read_excel('data/kaggle/the_office_series.xlsx')
series.rename(columns={'Season':'temporada',
                       'EpisodeTitle':'titulo',
                       'Ratings':'nota',
                       'Viewership':'audiencia',
                       'Duration':'duração',
                       'GuestStars':'estrelas_convidadas'}, inplace=True)

# juntando as informações e deletando colunas
eps = eps.merge(series, on='titulo', suffixes=('', '_delme'))
eps = eps[[c for c in eps.columns if not c.endswith('_delme')]]
del series

eps['titulo'] = eps['titulo'].astype('string')
eps['descricao'] = eps['descricao'].astype('string')
eps['dirigido_por'] = eps['dirigido_por'].astype('string')
eps['escrito_por'] = eps['escrito_por'].astype('string')
eps['estrelas_convidadas'] = eps['estrelas_convidadas'].astype('string')

# lendo falas e renomeando colunas
lines = pd.read_excel('data/kaggle/lines_official.xlsx')
lines.rename(columns={'season': 'temporada',
                     'episode': 'episodio',
                     'title': 'titulo',
                     'scene': 'cena',
                     'speaker': 'personagem',
                     'line_text': 'fala',
                     'deleted':'deletada'},
                     inplace=True)
lines = lines[['id', 'temporada', 'episodio', 'cena', 'personagem', 'fala', 'deletada']]

# limpando '[]'
lines['fala'] = lines['fala'].str.replace(r"[\(\[].*?[\)\]]", '', regex=True)
lines['fala'] = lines['fala'].str.replace("���", ' ', regex=True)
lines['fala'] = lines['fala'].str.replace("��", ' ', regex=True)
lines['fala'] = lines['fala'].str.replace("�", ' ', regex=True)
lines['fala'] = lines['fala'].str.strip()


lines = lines[lines['deletada'] == False]
lines['personagem'] = lines['personagem'].astype('string')
lines['fala'] = lines['fala'].astype('string')

lines['fala'].replace('', np.nan, inplace=True)
lines.dropna(subset=['fala'], inplace=True)
lines['n_palavras'] = [len(x.split()) for x in lines['fala'].tolist()]


# Adicionando mais informações para cada episodio
temporada = lines['temporada'].unique().tolist()
# temporada = [1]
ref = {
    'temporada':[],
    'episodio':[],
    'qtd_cena':[],
    'nº de falas':[],
    'nº de palavras':[],
    'nº de personagens dis':[],
    'quem_iniciou_eps':[],
    'quem_finalizou_eps':[],
    'lista_personagens':[]
}

for t in temporada:
        filt = lines[lines['temporada']==t]
        episodeos = filt['episodio'].unique().tolist()
        
        for e in episodeos:
            ult = filt[filt['episodio']==e]
            personagens = ult['personagem'].unique().tolist()
            personagens_ranking = ult['personagem'].tolist()

            for p in personagens:
                if '|' in p:
                    for i in p.split('|'):
                        personagens.append(i.strip())

            personagens = [ x for x in personagens if "|" not in x ]
            personagens = [j.strip() for j in personagens]
            personagens = list(dict.fromkeys(personagens))
            
            ref['temporada'].append(t)
            ref['episodio'].append(e)
            ref['qtd_cena'].append(ult['cena'].max())
            ref['nº de falas'].append(ult['fala'].count())
            ref['nº de palavras'].append(ult['n_palavras'].sum())
            ref['nº de personagens dis'].append(len(ult['personagem'].unique().tolist()))
            ref['lista_personagens'].append(str(personagens)[1:-1])
            ref['quem_iniciou_eps'].append(personagens[0])
            ref['quem_finalizou_eps'].append(personagens[-1])

plus_data = pd.DataFrame(ref)

eps = eps.merge(plus_data, on=['temporada', 'episodio'], suffixes=('', '_delme'))
eps = eps[[c for c in eps.columns if not c.endswith('_delme')]]
eps = eps.drop(columns=['id', 'Unnamed: 2', 'About'])


# Criando palavra quantidade
personagens_principais = ['Michael', 'Dwight', 'Jim', 'Pam', 'Ryan', 'Andy',
                        'Stanley', 'Phyllis', 'Creed', 'Meredith', 'Darryl',
                        'Angela', 'Oscar', 'Kevin',
                        'Toby', 'Kelly', 'Erin']

ref = {
    'quem_falou':[],
    'nome_falado':[],
    'qtd':[],
    'falando_proprio_nome':[]
}

async def quem_falou_quem(quem_falou, nome_falado):
    all_falas = []

    cut = lines[lines['personagem']==quem_falou]
    if cut.shape[0] <= 0:
        raise Exception('Nome não consta na base de dados')
    cut_l = cut['fala'].tolist()
    for f in cut_l:
        fala = re.sub('[^a-zA-Z]+', ' ', f).strip()
        for f_ in fala.split(' '):
            all_falas.append(f_.lower())

    counter = Counter(all_falas)
    palavras_qtd = pd.DataFrame(counter, index=[0])
    palavras_qtd = pd.melt(palavras_qtd, value_vars=palavras_qtd.columns.tolist())
    palavras_qtd.rename(columns={'variable': 'palavra', 'value': 'qtd'}, inplace=True)
    try:
        q_x_falou = palavras_qtd[palavras_qtd['palavra']==nome_falado.lower()]['qtd'].values[0]
    except:
        q_x_falou = 0

    return quem_falou, nome_falado, q_x_falou

for quem_falou in personagens_principais:
    for nome_falado in personagens_principais:
        q_f, n_f, q_x_f = await quem_falou_quem(quem_falou=quem_falou, nome_falado=nome_falado)
        ref['quem_falou'].append(q_f)
        ref['nome_falado'].append(n_f)
        ref['qtd'].append(q_x_f)
        ref['falando_proprio_nome'].append(1 if quem_falou == nome_falado else 0)
palavra_qtd = pd.DataFrame(ref)

# ordem
eps = eps[['temporada', 'episodio', 'episodeo_num_geral', 'titulo', 'descricao', 'dirigido_por',
           'escrito_por', 'data_de_estreia', 'Date', 'estrelas_convidadas', 'codigo', 'telespectadores_USA',
           'nota_imdb', 'nota', 'audiencia', 'duração', 'qtd_cena', 'nº de falas', 'quem_iniciou_eps',
           'quem_finalizou_eps', 'nº de palavras', 'nº de personagens dis', 'lista_personagens']]

eps = eps.rename(columns={
    'Date':'data_formatada',
    'nº de falas': 'n_falas',
    'nº de palavras':'n_palavras',
    'nº de personagens dis':'n_personagens_distintos'
})

palavra_qtd.to_excel('data/output_data/palavra_qtd.xlsx', index=False)
eps.to_excel('data/output_data/eps.xlsx', index=False)
lines.to_excel('data/output_data/lines.xlsx', index=False)

TypeError: str() takes at most 3 arguments (15 given)

In [3]:
p_inuteis = ['i', 'you', 'the', 'to', 'a', 's', 'it', 'that', 'and', 'is', 't', 'of', 'what', 'we', 'no', 'yes',
             'in', 'this', 'my', 'me', 'm', 'have', 'just', 'so', 'know', 'do', 'oh', 'not', 'for', 'on', 'don',
             'are', 're', 'be', 'was', 'your', 'can', 'he', 'she', 'like', 'all', 'with', 'okay', 'but', 'here',
             'right', 'get', 'about', 'there', 'well', 'hey', 'go', 'out', 'up']

palavras_qtd[ ~ palavras_qtd['palavra'].isin(p_inuteis)].sort_values(by='qtd', ascending=False)

,palavra,qtd
41,yeah,3094
120,think,2182
7,good,2117
55,michael,2036
221,one,2019
...,...,...
11729,retrospective,1
11730,realist,1
11734,matsuhashi,1
11736,excalibur,1


In [ ]:
summary = lines['fala']
all_summary = " ".join(str(s) for s in summary)
stopwords = set(STOPWORDS)
stopwords.update(['u', 'oh'])

wordcloud = WordCloud(stopwords=stopwords,
                      background_color='black', width=1600,
                      height=800).generate(all_summary)
fig, ax = plt.subplots(figsize=(16, 8))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()
plt.imshow(wordcloud)
wordcloud.to_file('WorldCloud.png')

In [ ]:
dataframefinal = []

for t in temporadas:
    
    all_falas = []
    lines_t = lines[lines['temporada']==t]
    falas = lines_t['fala'].tolist()
    for f in falas:
        fala = re.sub('[^a-zA-Z]+', ' ', f).strip()
        for f_ in fala.split(' '):
            all_falas.append(f_.lower())

    counter = Counter(all_falas)
    palavras_qtd = pd.DataFrame(counter, index=[0])
    palavras_qtd = pd.melt(palavras_qtd, value_vars=palavras_qtd.columns.tolist())
    palavras_qtd.rename(columns={'variable': 'palavra', 'value': 'qtd'}, inplace=True)
    palavras_qtd['temporada'] = t
    palavras_qtd = palavras_qtd[['temporada', 'palavra', 'qtd']]
    finar.append(palavras_qtd)
palavras_qtd = pd.concat(finar, ignore_index=True)

In [26]:
pd.DataFrame(ict)

,temporada,episodeo,nome,qtd
0,1,1,Michael,5
1,1,2,Michael,7
2,1,3,Michael,11
3,1,4,Michael,10
4,1,5,Michael,7
...,...,...,...,...
2785,9,19,Erin,0
2786,9,20,Erin,3
2787,9,21,Erin,2
2788,9,22,Erin,1
